In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

print('os name:', os.name)
print('python ver:', sys.version)
print('numpy ver:', np.__version__)
print('pandas ver:', pd.__version__)
print('matplotlib ver:', matplotlib.__version__)
print('seaborn ver:', sns.__version__)

os name: nt
python ver: 3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)]
numpy ver: 1.23.4
pandas ver: 1.5.0
matplotlib ver: 3.6.0
seaborn ver: 0.12.1


In [3]:
import sys
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor


from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix


In [4]:
plt.rc('font', family='Malgun Gothic') #한글 폰트 설정

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 30)
pd.set_option('display.max_seq_item', 500)

## 데이터 가공

In [ ]:
# 공통된 컬럼 찾기
set(df_y.columns)&set(df_x.columns)

# x_train, y_train 합치기
df = pd.merge(df_x, df_y, on=['D3', 'D4', 'D5'], how='left')

### 정보 조회

In [ ]:
df.describe()
data.info(verbose=True, null_counts=True)

### Target Data 균등 분포 확인(분류 분석 시)
- 만약 불균등이라면 SMOTE 방식으로 오버샘플링 처리 필요

In [ ]:
df['Target'].value_counts(normalize=True)

### 모든 값이 동일한 열 제거

In [ ]:
#1 열 이름 파악
df.nunique()

#2 제거
A = df.loc[:,df.nunique()==1].columns
df = df.drop(columns=A)

### 결측치 처리

- 열 별 결측치 비율 별 처리 방안

![](2022-10-28-00-12-43.png)

In [ ]:
# 결측치 파악 : 중간에 계속 해보면서 NaN이 남아있는 행 파악
df.isna().sum().sort_values("ascending=False")

# 결측치 시각화(결측치가 아닌 값들을 보여줌)
!pip install missingno
import missingno as msno
missingno.bar(ldf, figsize=(10,5), fontsize=12, sort="ascending")

# 결측치가 존재하는 컬럼명 확인
df = df.loc[:,  df.isna().sum() > 0 ].columns.values

# 결측치 제거
df.drop(['A'], axis=1) #특정 컬럼 제거
df = df.dropna(axis=0, thresh=10) #NaN 값이 10개 이상인 행 삭제

# 결측치 치환(연속형)
df = df.fillna(value=5, axis=0, method = None) #bfill, ffill도 가능

df = df['X2'].fillna(value=df['X2'].mean()) #평균값으로 치환
df['X2'].describe() #치환 후 Data 통계량 확인

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='constant', fill_value=0) #0으로 치환
A = imputer.fit_transform(df[['Age', 'Pclass']]) #치환이 필요한 컬럼
df['Age','Pclass'] = A #원래 컬럼에 대입

# 결측치 치환(범주형)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'most_frequent') #최빈값으로 치환
result = imputer.fit_transform(train[['Embarked', 'Cabin']])
train[['Embarked', 'Cabin']] = result

# 특정 조건 만족하는 열 제거
df=df.drop(columns=df.loc[:,df.var()==0].columns)

# 그 외 불필요한 열 삭제(data_no나 ID 같은 것들)
df = df.drop(['data_no', 'Id'], axis=1)

### 상관 관계 분석
- 높은 상관관계를 갖는 독립변수를 찾아서 제거 또는 축소(PCA : 주성분 분석)
- 열 제거 시 짝이 되는 독립 변수 중 Target과 Corr이 더 작은 독립변수 삭제

In [ ]:
# 데이터 분포 및 변수간 관계 분석(오래 걸림)
sns.pairplot(df)
#sns.pairplot(iris, vars=['sepal_width', 'sepal_length']) #vars를 사용해 특정 변수만 지정 가능

plt.show()

# 모든 Data 간 선형관계 확인
df.corr(numeric_only=True).round(3)
# 만약 맨 오른쪽에 독립변수만 object이고 나머지는 다 수치형일 경우: df.iloc[:, :-1].corr() 

sns.heatmap(df.corr(numeric_only=True).round(2), vmin=-1, cmap='RdBu_r', annot=True)

# 독립변수와 종속변수 간 선형관계만 따로 확인(이때 상관관계가 너무 큰 컬럼은 제거 위해)
B= df.corr(numeric_only=True)['Y']
B = B.loc[B.index!='Y']
B

# 시각화(독립변수와 종속변수 간 선형관계)
sns.barplot(x=B.index, y=B.values)
plt.xticks(rotation=45)
plt.show()

# 컬럼 별로 Corr이 가장 높은 행에 상관관계, 여기서 1이랑 근접한 열 파악
for idx in range(len(df.corr())):
    s = df.corr().iloc[idx]
    print(s.drop(s.index[idx]).max())

# 컬럼 별로 Corr이 가장 높은 행에 상관관계, 여기서 -1이랑 근접한 열 파악
for idx in range(len(df.corr())):
    s = df.corr().iloc[idx]
    print(s.drop(s.index[idx]).min())

### 연속형 자료 분석(이상치 확인)

TIP
1) 단일변수 백분위에서 1%단위마다 위치하는 값 표시(변수.quantile(np.arange(0,1,0.01))
2) 단일변수 MIN, MAX값 확인
3) 단일변수 히스토그램 확인(크게 벗어난 위치 확인)

처리방법
1) 단순 삭제 : Human Error 또는 오타에 의한 것이라면 행 삭제
2) 다른 값으로 대체 : Data 양이 적을 경우 이상체 제거보다 다른 값(평균, 중간값 등)으로 치환
ex) 엄청 큰 값이 1,2개 정도면 다음 작은 Max값으로 대체, 0이 나올 수 없는 수치들은 다음 Min값으로 대체

In [ ]:
#1 Boxplot을 통해 이상치가 포함되어 있는 열 이름 파악
df.plot.box()

#2 Subplot으로 더 자세히 확인
idx = df.loc[:, 'Wafer_Site':'X19']
row = 5, column = 4
plt.figure(figsize=(9, 9))

for i, v in enumerate(idx):
    plt.subplot(row, column, i+1)
    sns.boxplot(y=v, data=df)
plt.tight_layout()

#3 백분위 별 위치하는 값 확인
df['X'].quantile(np.arange(0,1,0.01))

#4 단일 변수 Min, Max값 확인
df.agg(['max','min'])

#5 단인 변수 histogram에서 Data 분포 확인 (나중에 StandardScaler 쓸 때 Outlier가 있으면 성능 저하)
plt.hist(df['X'], bins=50)

#6 IQR을 사용하여 이상치 제거(변수 별로 봤을 때 이상치에 해당하는 값의 개수가 적으면 사용 자제 - 너무 큰 변환이 됨)
Q1, Q3 = df['X'].quantile([0.25, 0.75]) # 이상치 제거하고자 하는 열 입력
IQR = Q3-Q1
lower_f = Q1 - 1.5*IQR
upper_f = Q3 + 1.5*IQR
outlier = df.loc[(df['X'] < lower_f) | (df['X'] > upper_f), :]
outlier.shape  # 해당되는 Row수 확인, 전체 Row 수 대비 많이 안 지워지는지 확인

df_wo_outlier = df.drop(outlier.index, axis=0)
df_wo_outlier.shape # 행 삭제 후 비교

### 범주형 자료 분석

In [ ]:
# groupby 함수를 활용하여 범주형 자료 별 통계 분석
df.groupby(by='A')[['B','C']].agg(['sum', 'mean']) 
df.groupby(by='A')['B'].quantile(0.75) # 'A' 컬럼에 존재하는 각 변수값 별로 'B' 컬럼에 대해 상위 25%

# pitovt_table을 활용하여 열 별 관계 파악
df.pivot_table(index=['A','B'], columns='D', values=['C', 'D'], aggfunc='mean', fill_value=0)

# 위에서 범주형 자료에 관계를 토대로 한 변수에 지나치게 의존적이면 삭제 

# Statistically Significant가 나오면 변수로 넣어주자

##### 범주형 - 범주형 두개의 변수는 독립일까? 관계가 있을까?
##### 카이제곱 검정
##### 귀무가설 : 두 변수의 관계는 독립이다
##### 대립가설 : 두 변수의 관계는 독립이 아니다
##### p ≤ 0.05 일때 두 변수는 독립이 아닌 관계가 있는 것
from scipy import stats

def statistical_test(mode, df, cat, target):
    uniques= df[cat].unique()    
    if mode == 't':   # 범주가 1개, 2개 일때 사용 - 범주별 평균의 차이가 유의미한가? (성별에 따른 키 평균)
        group = df.groupby(cat)[target]
        samples = [group.get_group(i) for i in uniques]
        value, p = stats.ttest_ind(*samples)        
    elif mode == 'f':   # 범주가 3개 이상일 때 사용 - 범주별 평균의 차이가 유의미한가? (혈액형에 따른 키 평균)
        group = df.groupby(cat)[target]
        samples = [group.get_group(i) for i in uniques]
        value, p = stats.f_oneway(*samples)
    elif mode == 'c':   # 두 개 범주가 독립성을 갖는지 아닌지? (가사노동의 종류와 구성원은 독립적? 연관성?)
        contingency = pd.crosstab(index=df[cat], columns=df[target])
        value, p, df, expected = stats.chi2_contingency(contingency)
        
    return value, p, 'Statistically Signifcant' if p<=0.05 else 'Statistically Insignifcant'

In [8]:
# 유지해야할 범주형 Data가 있다면 아래 방법들로 수치형으로 변환 필요(사전에 결측치 제거 필요)

#1 Label Encoding - 명목형 데이터(성별, 혈액형, 국가, 지역, 부서)에 사용 부적합
# 이때 숫자로 바꿔버리면 문자처럼 1대1 대응 방식으로 학습하는게 아니라 숫자들의 크기가 생겨서 관계까지 학습해버려서 안 좋은 결과가 나올 수 있음
# 그래서 이건 순서가 의미가 있을 때 사용하는게 좋음(계급, 직급, 점수 등등)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['X'] = le.fit_transform(df['X']) # 변환하고자 하는 문자열 컬럼 대입
df['X'].value_counts() #제대로 변환 됐는지 확인

le.inverse_transform(df['X']) # 나중에 역변환 시 사용

#2 One-Hot Encoding - 명목형 데이터와 더 잘 어울림
df = pd.get_dummies(df)

#기타 s.astype('category').cat.codes 사용법 : 눈에 보일때는 문자열이지만 실제 저장된 것은 숫자
s = pd.Series(list("ABCAABABCC"))
s = s.astype('category').cat.codes
s.info(), s

<class 'pandas.core.series.Series'>
RangeIndex: 10 entries, 0 to 9
Series name: None
Non-Null Count  Dtype
--------------  -----
10 non-null     int8 
dtypes: int8(1)
memory usage: 138.0 bytes


(None,
 0    0
 1    1
 2    2
 3    0
 4    0
 5    1
 6    0
 7    1
 8    2
 9    2
 dtype: int8)

## 독립(X)/종속(Y)변수 분리

In [ ]:
# X
X = df.iloc[:, :-1]
X = df.loc[:,df.columns != '종속 변수']

# Y
Y = df.iloc[:, -1] 
Y = df.loc[:,'종속 변수']

print(X.shape, Y.shape)  # Row 개수가 동일하지 확인 (Row, Col)

## 스케일링

In [ ]:
# Normalize
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Standardize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Polynomial Features
from sklearn.preprocessing import PolynomialFeatures
degree = 3
P = PolynomialFeatures(degree=degree, include_bias=False)
X_poly = P.fit_transform(X)

## 데이터 분할

In [ ]:
- test_size = 0.25: 0.0~1.0 테스트 데이터셋 비율
- train_size = None: 0.0~1.0 훈련 데이터셋 비율, test_size를 작성하면 작성하지 않아도 자동 산출
- random_state = None: 정수 값, 난수 발생의 시드(seed) 값
- stratify: y의 지정한 데이터 비율을 유지(층화 추출), y가 범주형일 때 사용함!!!!! 연속형일 때는 쓰지 말자

from sklearn.model_selection import train_test_split

# Scaler 미사용 시
A = train_test_split(X, Y, random_state=0, test_size=0.25, stratify=y)

# Scaler 사용 시 X_scaled 확인 필요
A = train_test_split(X_scaled, Y, random_state=0, test_size=0.25, stratify=y)  

# PolynomialFeatures 사용 시 X_scaled 확인 필요
A = train_test_split(X_poly, Y, random_state=0, test_size=0.25) 


x_train, x_test, y_train, y_test = A

[x.shape for x in [x_train, x_test, y_train, y_test]]  #데이터 분할된 비율이 제대로 됐는지 확인 목적


## 베스트 알고리즘 출력 함수

In [ ]:
Best_Algorithm = {'Algorithm': 'name', 'TrainScore': 0, 'TestScore': 0}
Name = []
TrainScore = [] #각각의 Score를 알고리즘 별로 수집해놓기
TestScore = []

def BestAlgorithm(name, trainScore, testScore):
    print()
    print("현재까지 Best Algorithm")
    if Best_Algorithm['TestScore'] < testScore:
        Best_Algorithm['Algorithm'] = name
        Best_Algorithm['TrainScore'] = trainScore
        Best_Algorithm['TestScore'] = testScore
    
    Name.append(name)
    TrainScore.append(trainScore)
    TestScore.append(testScore)
    
    return Best_Algorithm

## 선형 모델링(분류-범주형)

- max_iter = 100: Maximum number of iterations taken for the solvers to converge. Tolerance값보다 작으면 max_iter 전에 빠져나옴
    - Undefitting일 경우 크게, overfitting일 경우 작게 설정
- tol = 0.0001: Tolerance for stopping criteria.
    - Overfitting일 경우 크게 설정
- C =1: Inverse of regularization strength; must be a positive float, 규제 강도인 alpha와 반대 개념
    - Overfitting일 경우 작게 설정

In [ ]:
# LogisticRegression
from sklearn.linear_model import LogisticRegression
model_LR = LogisticRegression(penalty='l2', C=1.0, tol=0.0001, random_state=None, max_iter=100).fit(x_train,y_train)

# Decision Tree
from sklearn.tree import DecisionTreeClassifier
tree_gini = DecisionTreeClassifier(criterion='gini', random_state=0).fit(x_train, y_train)
tree_ent = DecisionTreeClassifier(criterion='entropy', random_state=0).fit(x_train, y_train)
  # criterion : 'gini'(default), 'entropy'
  # max_depth = None, 트리의 최대 깊이
  # min_sample_split = 2, 노드 내에서 분할이 필요한 최소의 샘플 수
  # max_features = None, 최적의 분할을 찾기 위해 고려할 Feature의 수
  # max_leaf_nodes=None : 최대 말단 노드 개수

# Random Forest
from sklearn.ensemble import RandomForestClassifier
model_RFR = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2).fit(x_train, y_train)

# Light GBM
from lightgbm import LGBMClassifier
model = LGBMClassifier(n_estimators=1000, max_depth=3, num_leaves=31, min_data_in_leaf=20).fit(x_train, y_train)  
#과적합시 max_depth, min_data_in_leaf, num_leaves 줄이기

# XGB Classifier
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators=100, learning_rate=0.4, max_depth=3, objective ='reg:squarederror').fit(x_train, y_train)

# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
model_GBR = GradientBoostingClassifier(learning_rate=0.1, n_estimators=100, max_depth=3).fit(x_train, y_train)


# Voting Classifier
from sklearn.ensemble import VotingClassifier
names = ['knn3', 'knn5', 'lr', 'dt3', 'dt5']
estimator = [knn3, knn5, lr, dt3, dt5]
estimators = list(zip(names,estimator))

hard = VotingClassifier(estimators, voting='hard') #hard
soft = VotingClassifier(estimators, voting='soft') #soft

# KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

k_range = range(1, 11)
train_score = []
test_score = []
for k in k_range:
    model = KNeighborsClassifier(n_neighbors=k).fit(x_train, y_train)
    train_score.append(model.score(x_train, y_train))
    test_score.append(model.score(x_test, y_test))
print(train_score)
print(test_score) 

# kNN 시각화
plt.plot(k_range, train_score, 'ro-', label='Train Accuracy')
plt.plot(k_range, test_score, 'bo--', label='Test Accuracy')
plt.xticks(k_range)
plt.title('Find Best K-value')
plt.legend(loc='best')
plt.grid(True)
plt.show()

## 선형 모델링(회귀-연속형)
- alpha(규제 강도): Overfitting 발생 시 모델을 단순화(=규제) 시키기 위한 정도, alpha가 너무 크면 과소적합, 너무 작으면 과대적합

In [ ]:
# LinearRegression
from sklearn.linear_model import LinearRegression  # y1 = a*x1 + b
model_LR = LinearRegression().fit(x_train, y_train)

# Lasso
from sklearn.linear_model import Lasso  # L1, 가중치의 절댓값의 합이 최소가 되도록 설정, alpha가 커질수록 사용되는 X 종류가 줄어듬, 사전에 Scaling 필요
alpha = 1 
model_lasso = Lasso(alpha=alpha).fit(x_train, y_train)

# Ridge
from sklearn.linear_model import Ridge  # L2, 가중치의 제곱합이 최소가 되도록 설정, 특정 가중치로 쏠리는 형상 방지, 사전에 Scaling 필요
alpha = 1 
model_ridge = Ridge(alpha=alpha).fit(x_train, y_train)

# DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor
model_DTR = DecisionTreeRegressor(max_depth = None, min_samples_split = 2).fit(x_train, y_train)

# RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
model_RFR = RandomForestRegressor(n_estimators=100, max_depth=None, min_samples_split=2).fit(x_train, y_train)

# GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
model_GBR = GradientBoostingRegressor(learning_rate=0.1, n_estimators=100, max_depth=3).fit(x_train, y_train)

# XGB Regressor
from xgboost import XGBRegressor
model_xgb = XGBRegressor(n_estimators=100, learning_rate=0.4, max_depth=3, objective ='reg:squarederror').fit(x_train, y_train)

# LightGBMRegressor
from lightgbm import LGBMRegressor
model = LGBMRegressor(n_estimators=1000, max_depth=3, num_leaves=31, min_data_in_leaf=20).fit(x_train, y_train)  
#과적합시 max_depth, min_data_in_leaf, num_leaves 줄이기

# KNeighborRegressor
from sklearn.neighbors import KNeighborsRegressor
k_range = range(1, 11)
train_score = []
test_score = []

Max_test = 0 

for k in k_range:
    model_knn = KNeighborsRegressor(n_neighbors=k).fit(x_train_k, y_train_k)
    trainScore_k = model_knn.score(x_train_k, y_train_k)
    testScore_k = model_knn.score(x_test_k, y_test_k)
    
    if testScore_k > Max_test:
        good_train = trainScore_k
        good_test = testScore_k
        good_k = k
        
    train_score_knn.append(trainScore_k)
    test_score_knn.append(testScore_k)

print(f'trainScore : {good_train}')
print(f'testScore : {good_test}')

# kNN 시각화
plt.plot(k_range, train_score, 'ro-', label='Train Accuracy')
plt.plot(k_range, test_score, 'bo--', label='Test Accuracy')
plt.xticks(k_range)
plt.title('Find Best K-value')
plt.legend(loc='best')
plt.grid(True)
plt.show()


In [ ]:
#GridSearchCV
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier

# 객체 생성
model_xgb = XGBClassifier(n_estimators=100, random_state=0)

# 후보 파라미터 선정
params = {'max_depth':[5,7], 'min_child_weight':[1,3], 'colsample_bytree':[0.5,0.75]}

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번) 및 튜닝 시작
gs_xgb = GridSearchCV(model_xgb, param_grid=params, cv=3).fit(x_train, y_train)

#튜닝된 파라미터 출력
print(gs_xgb.best_params_)

#최고 점수 출력
print(gs_xgb.best_score_)

## 비선형 모델링

In [ ]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

## 모델 평가

In [ ]:
print(f'train score : {model.score(x_train, y_train):.3f}') 
print(f'test score  : {model.score(x_test, y_test):.3f}')


# 함수용
trainScore = round(model.score(x_train, y_train),3)
testScore = round(model.score(x_test, y_test),3)
print('train_score:', trainScore_LR, 'test_score:', testScore_LR)

## 모델 예측

In [ ]:
#실제값
y_test

#예측값
model.predict(x_test)

# 처음 5개만 Test 하고 싶을 때
y_test.iloc[:5].to_numpy() #실제값(0~4번째 컬럼)
model.predict(x_test[:5]) #에측값

# 분류 모델일 경우, 범주 별 예측 확률을 출력해주며 가장 높은 확률을 가진 것이 모델의 예측값으로 출력됨
model.predict_proba(x_test)  

## 함수 출력

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(Name, TrainScore, label='TrainScore')
plt.plot(Name, TestScore, label='TestScore')
plt.xlabel('Algorithm')
plt.ylabel('Score')
plt.ylim(0.5,1.05)
plt.grid()
plt.title('Find Best Algorithm')
plt.legend(loc='lower right',fontsize='x-large')

## 성능 평가

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def print_lr_errors(model, X_test, y_test):
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)  # = mse**0.5
    r2 = r2_score(y_test, pred)
    print(f'MAE:{mae:.3f}, MSE:{mse:.3f}, RMSE:{rmse:.3f}, R2:{r2:.3f}')


print_lr_errors(model, x_train, y_train)
print_lr_errors(model, x_test, y_test)

In [ ]:
#평가 지표
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, roc_auc_score

def metrics(y_test,pred):
    accuracy = accuracy_score(y_test,pred) #전체에서 맞춘 것의 비율
    precision = precision_score(y_test,pred) #예측한 것 중에 맞춘 것
    recall = recall_score(y_test,pred) # 실제인 것 중에 맞춘 것
    f1 = f1_score(y_test,pred) # precision과 recall의 조화 평균
    roc_score = roc_auc_score(y_test,pred,average='macro')
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy,precision,recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1,roc_score,recall))

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
import numpy as np
import pandas as pd

y_true = np.array([0, 1, 0, 0])   # 0 : 3개, 1 : 1개, 불균형한 데이터인 경우 
y_pred = np.array([0, 1, 1, 0])
cm = pd.DataFrame(confusion_matrix(y_true, y_pred))
print(cm)
# 전체에서 맞춘것의 비율 : 3/4
print('accuracy  : %.2f' % accuracy_score(y_true, y_pred))
# positive(1) 로 예측한 것 중에 맞춘 것 : 1/2
print('precision : %.2f' % precision_score(y_true, y_pred))
# positive(1) 가 실제인 것 중에 맞춘 것 : 1/1
print('recall    : %.2f' % recall_score(y_true, y_pred))
# f1 : precision, recall 의 조화 평균
print('f1        : %.2f' % f1_score(y_true, y_pred))

In [ ]:
# ROC curve 그래프에 필요한 데이터 준비
from sklearn.metrics import roc_curve, auc, roc_auc_score

y_pred_proba = model.predict_proba(x_test)[:,1]
FP, TP, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(FP, TP)  

#roc_auc = roc_auc_score(y_test, y_pred_proba)  # 동일 방법
print('AUC : %.3f' % (roc_auc))
print(thresholds)

In [ ]:
from sklearn.metrics import classification_report

y_train_pred = model.predict(x_train)
A = classification_report(y_train, y_train_pred, target_names=["Fail", "Pass"])
print(A)

In [ ]:
from sklearn.model_selection import train_test_split
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=10)

#모델링
def modeling(model,x_train,x_test,y_train,y_test):
    model.fit(x_train,y_train)
    pred = model.predict(x_test)
    metrics(y_test,pred)